In [ ]:
import pandas as pd
import re
import numpy as np
import nltk
import spacy
import string
from emot.emo_unicode import UNICODE_EMO, EMOTICONS
import functools
import operator
import emoji

In [ ]:
#Importing stopwords from nltk library
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

In [ ]:
# add space near your emoji
    def add_space(text):
        return ''.join(' ' + char if is_emoji(char) else char for char in text).strip()

In [ ]:
# search your emoji    
    def is_emoji(s):
        return s in UNICODE_EMO

In [ ]:
# Function for converting emoticons into word
def convert_emoticons(text):
    for emot in EMOTICONS:
        text = re.sub(u'('+emot+')', "_".join(EMOTICONS[emot].replace(",","").split()), text)
    return text

In [ ]:
# Function for converting emojis into word
def convert_emojis(text):
    for emot in UNICODE_EMO:
        text = text.replace(emot, "_".join(UNICODE_EMO[emot].replace(","," ").replace(":"," ").split()))
    return text

In [ ]:
# Function to remove the stopwords
def stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

In [ ]:
from bs4 import BeautifulSoup
#Function for removing html
def html(text):
    return BeautifulSoup(text, "lxml").text

In [ ]:
def cleaner(data):
    '''
    Riceve i dati di reddit e li pulisce
    @params:
        data:   Dati su reddit in formato csv
    '''
    # Importazione dei dati
    df = pd.read_csv(data, sep = ",")
    
    # Colonne utili
    df= df[['Title']
    
    #Sostituisco spazi multipli con singoli spazi
    df['Title'] = df['Title'].apply(lambda x:re.sub(r'\s+', ' ', x, flags=re.I))
    
    #Elimino spazi iniziali
    df['Title']= df['Title'].apply(lambda x:re.sub(r"^ ", "", x))
    
    #Elimino spazi finali
    df['Title']= df['Title'].apply(lambda x:re.sub(r" $", "", x))
    
    #Aggiungo spazi tra Emoji
    df['Title'] = df['Title'].apply(add_space)

    #Converting Emoji and Emoticons to words
    df['Title'] = df['Title'].apply(convert_emoticons)
    df['Title'] = df['Title'].apply(convert_emojis)

    #Remove all URLS
    df['Title']= df['Title'].apply(lambda x:re.sub(r"http\S+", "", x))
    
    #Lower Casing
    df['Title']  = df['Title'].apply(lambda x: x.lower())
    
    #Removal of Punctuations
    df['Title']= df['Title'].apply(lambda x:re.sub(r"[^\w\s]", "", x))
    
    #Stop-word removal
    # Applying the stopwords to 'text_punct' and store into 'text_stop'
    df["Title"] = df["Title"].apply(stopwords)
    
    #Removal of HTML tags
    df['Title'] = df['Title'].apply(html)
    
    #Creo un nuovo dataset pulito
    df.to_csv('reddit_pulito.csv')